In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score, roc_auc_score
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.preprocessing import LabelEncoder
# Applying BoW
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split



In [2]:
temp_df = pd.read_csv('Suicide_Detection.csv')

In [3]:
df = temp_df.iloc[:20000]

In [4]:
df.head()

,Unnamed: 0,text,class
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,3,Am I weird I don't get affected by compliments...,non-suicide
2,4,Finally 2020 is almost over... So I can never ...,non-suicide
3,8,i need helpjust help me im crying so hard,suicide
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


In [5]:
df.drop(['Unnamed: 0'],axis=1, inplace=True)

C:\Users\Ulka\AppData\Local\Temp\ipykernel_4548\1160589198.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Unnamed: 0'],axis=1, inplace=True)


In [6]:
df.head()

,text,class
0,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,Am I weird I don't get affected by compliments...,non-suicide
2,Finally 2020 is almost over... So I can never ...,non-suicide
3,i need helpjust help me im crying so hard,suicide
4,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


In [7]:
df['text'] = df['text'].apply(lambda x : x.lower())

C:\Users\Ulka\AppData\Local\Temp\ipykernel_4548\3534604001.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(lambda x : x.lower())


In [8]:
# Download necessary resources from NLTK
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ulka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ulka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
import nltk
from nltk import word_tokenize, pos_tag

# Download NLTK POS tagger and tokenizer if you haven't
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Ulka\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ulka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:


# # Define POS tags that are important for depression detection
# IMPORTANT_POS = {"JJ", "JJR", "JJS", "RB", "RBR", "RBS", "PRP", "PRP$", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ"}

# def filter_important_pos_nltk(text):
#     # Tokenize the text
#     tokens = word_tokenize(text)
    
#     # Get POS tags for each token
#     tagged_tokens = pos_tag(tokens)
    
#     # Keep only tokens that have important POS tags
#     filtered_tokens = [word for word, pos in tagged_tokens if pos in IMPORTANT_POS]
    
#     # Return filtered tokens as a string or a list
#     return " ".join(filtered_tokens)


# df['text'] = df['text'].apply(
#     lambda x: filter_important_pos_nltk(x)
# )

# df.head()


In [11]:
# Function to remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

# Function to remove stop words
def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    return [word for word in tokens if word.lower() not in stop_words]

# Function to tokenize text
def tokenize(text):
    return word_tokenize(text)

# Additional cleaning: remove non-alphabetic tokens and convert to lowercase
def clean_tokens(tokens):
    return [word.lower() for word in tokens if word.isalpha()]

# Apply all steps: remove punctuation, tokenize, clean tokens, and remove stop words
df['text'] = df['text'].apply(
    lambda x: clean_tokens(remove_stopwords(tokenize(remove_punctuation(x))))
)

# Output the cleaned and tokenized DataFrame
df.head()

C:\Users\Ulka\AppData\Local\Temp\ipykernel_4548\2078368733.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(


,text,class
0,"[ex, wife, threatening, suiciderecently, left,...",suicide
1,"[weird, dont, get, affected, compliments, comi...",non-suicide
2,"[finally, almost, never, hear, bad, year, ever...",non-suicide
3,"[need, helpjust, help, im, crying, hard]",suicide
4,"[losthello, name, adam, struggling, years, afr...",suicide


In [12]:
# Initialize the Porter Stemmer
stemmer = PorterStemmer()

# Function to apply stemming
def stem_tokens(tokens):
    return [stemmer.stem(word) for word in tokens]

# Apply the cleaning functions and stemming to the 'tokenized_review' column
df['text'] = df['text'].apply(stem_tokens)

# Output the cleaned and stemmed DataFrame
df.head()


C:\Users\Ulka\AppData\Local\Temp\ipykernel_4548\847003611.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(stem_tokens)


,text,class
0,"[ex, wife, threaten, suiciderec, left, wife, g...",suicide
1,"[weird, dont, get, affect, compliment, come, s...",non-suicide
2,"[final, almost, never, hear, bad, year, ever, ...",non-suicide
3,"[need, helpjust, help, im, cri, hard]",suicide
4,"[losthello, name, adam, struggl, year, afraid,...",suicide


In [13]:
# Join the tokenized lists into strings for CountVectorizer
df['text'] = df['text'].apply(lambda tokens: ' '.join(tokens))


C:\Users\Ulka\AppData\Local\Temp\ipykernel_4548\681359426.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(lambda tokens: ' '.join(tokens))


In [14]:
X = df.iloc[:,0:1]
y = df['class']

In [15]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [16]:
#suicide=1, non-suicide=0
y

array([1, 0, 0, ..., 1, 1, 0])

In [17]:

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [18]:
X_train.shape

(16000, 1)

In [19]:
# For unigrams and bigrams
cv2 = CountVectorizer(ngram_range=(1, 2), max_features=2000)  # This includes both unigrams and bigrams  #20000 overall accuracy increased
X_train_bow2 = cv2.fit_transform(X_train['text']).toarray()
X_test_bow2 = cv2.transform(X_test['text']).toarray()


In [20]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf.fit(X_train_bow2,y_train)
y_pred = rf.predict(X_test_bow2)
accuracy_score(y_test,y_pred)

0.8695

In [21]:
confusion_matrix(y_test,y_pred)

array([[1733,  304],
       [ 218, 1745]], dtype=int64)

In [22]:
# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print results
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

print(classification_report(y_test, y_pred))

Accuracy: 0.87
Precision: 0.85
Recall: 0.89
F1 Score: 0.87
              precision    recall  f1-score   support

           0       0.89      0.85      0.87      2037
           1       0.85      0.89      0.87      1963

    accuracy                           0.87      4000
   macro avg       0.87      0.87      0.87      4000
weighted avg       0.87      0.87      0.87      4000



In [23]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer


# # Define POS tags that are important for depression detection
# IMPORTANT_POS = {"JJ", "JJR", "JJS", "RB", "RBR", "RBS", "PRP", "PRP$", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ"}

# def filter_important_pos_nltk(text):
#     # Tokenize the text
#     tokens = word_tokenize(text)
    
#     # Get POS tags for each token
#     tagged_tokens = pos_tag(tokens)
    
#     # Keep only tokens that have important POS tags
#     filtered_tokens = [word for word, pos in tagged_tokens if pos in IMPORTANT_POS]
    
#     # Return filtered tokens as a string or a list
#     return " ".join(filtered_tokens)




# Initialize the Porter Stemmer
stemmer = PorterStemmer()

# Function to preprocess a single text entry
def preprocess_text(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Tokenize text
    tokens = word_tokenize(text)
    
    # Remove stop words and apply stemming
    stop_words = set(stopwords.words('english'))
    cleaned_tokens = [stemmer.stem(word.lower()) for word in tokens if word.isalpha() and word.lower() not in stop_words]
    
    # Join the cleaned tokens into a single string
    return ' '.join(cleaned_tokens)

# Example usage with a new text entry
new_data = '''Sure! Here's a more upbeat and positive WhatsApp-style conversation:

---

**[10:15 AM]**
*You*: Hey! Guess what!

**[10:17 AM]**
*Friend*: Hey! What’s up? What happened?

**[10:18 AM]**
*You*: I just got the job offer I’ve been waiting for! 🎉

**[10:19 AM]**
*Friend*: OMG! That’s amazing! Congratulations!! 🥳 I knew you’d get it!!

**[10:20 AM]**
*You*: Thanks! I’m so excited, I can’t believe it finally happened! 😄

**[10:21 AM]**
*Friend*: You totally deserve it! All your hard work finally paid off. So when do you start?

**[10:22 AM]**
*You*: In two weeks! I already signed the offer letter. Just gotta wrap up some things at my current job.

**[10:23 AM]**
*Friend*: That’s so great! We need to celebrate! 🎉 Dinner this weekend?

**[10:24 AM]**
*You*: Definitely! Let’s do it! I’m still in shock, haha.

**[10:25 AM]**
*Friend*: I’m so happy for you!! This is just the beginning. You’re going to crush it at your new job!

**[10:27 AM]**
*You*: Thanks! I’m a bit nervous but mostly just excited to start something new. It’s going to be such a big change.

**[10:28 AM]**
*Friend*: It’ll be a great change, trust me. You’re going to do awesome! So proud of you! ❤️

**[10:29 AM]**
*You*: You’re the best! Can’t wait for dinner this weekend!

---

This conversation is filled with excitement and positivity around a great achievement!'''.lower()

# Preprocess the new data
new_data = preprocess_text(new_data)

# Output the cleaned text
print(new_data)








# Create a DataFrame for the new data
data1 = pd.DataFrame({'text': [new_data]})

# Transform the new data using the fitted vectorizer
X1 = cv2.transform(data1['text']).toarray()  # Correctly access the 'text' column
X1.shape









# Make predictions on the new data
new_predictions = rf.predict(X1)
print(new_predictions)

# Make predictions on the new data
new_probabilities = rf.predict_proba(X1)

# Display the results
for text, probabilities in zip(data1['text'], new_probabilities):
    # Extract the probability of the positive class (class 1)
    probability_suicide = probabilities[1]  # Probability of class 1 (suicide)
    label = "suicide" if probability_suicide >= 0.5 else "non-suicide"
    
    # Print text with predicted label and probability
    print(f'Text: "{text}" - Predicted Label: {label}, Probability of Suicide: {probability_suicide:.4f}')





# # Display the results
# for text, prediction in zip(data1['text'], new_predictions):
#     label = "suiside" if prediction == 1 else "non-suicide"
#     print(f'Text: "{text}" - Predicted Label: {label}')


sure here upbeat posit whatsappstyl convers hey guess friend hey happen got job offer wait friend omg amaz congratul knew get thank excit believ final happen friend total deserv hard work final paid start two week alreadi sign offer letter got ta wrap thing current job friend great need celebr dinner weekend definit let still shock haha friend happi begin go crush new job thank bit nervou mostli excit start someth new go big chang friend great chang trust go awesom proud best wait dinner weekend convers fill excit posit around great achiev
[0]
Text: "sure here upbeat posit whatsappstyl convers hey guess friend hey happen got job offer wait friend omg amaz congratul knew get thank excit believ final happen friend total deserv hard work final paid start two week alreadi sign offer letter got ta wrap thing current job friend great need celebr dinner weekend definit let still shock haha friend happi begin go crush new job thank bit nervou mostli excit start someth new go big chang friend g

In [24]:
# import pickle

# # Save the model
# with open('model.pkl', 'wb') as model_file:
#     pickle.dump(rf, model_file)

# # Save the vectorizer
# with open('vectorizer.pkl', 'wb') as vectorizer_file:
#     pickle.dump(cv2, vectorizer_file)  # Assuming 'cv2' is your vectorizer (CountVectorizer or TfidfVectorizer)
